# Müşteri Segmantasyonu

Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor. Buna yönelik olarak müşterilerin davranışlarını tanımlayacağız ve bu davranışlara göre gruplar oluşturacağız. Bu gruplara özel satış ve pazarlama teknikleri geliştirmeye çalışacağız.


**Değişkenler** <br>
    &emsp;**InvoiceNo**: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder <br>
    &emsp;**StockCode**: Ürün kodu. Her bir ürün için eşsiz numara <br>
    &emsp;**Description**: Ürün ismi <br>
    &emsp;**Quantity**: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir <br>
    &emsp;**InvoiceDate**: Fatura tarihi ve zamanı <br>
    &emsp;**UnitPrice**: Ürün fiyatı (Sterlin cinsinden)<br>
    &emsp;**CustomerID**: Eşsiz müşteri numarası<br>
    &emsp;**Country**: Müşterinin yaşadığı ülke<br>
    

## Veriyi Anlama ve Hazırlama

In [1]:
#Gerekli importlar
import datetime as dt
import pandas as pd

#Tüm sütunları görmek üzere ayarlandı.
pd.set_option('display.max_columns', None)

#Virgülden sonraki basamak sayısı 2 olarak ayarlandı.
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
#Veri seti okuma işlemi yapıldı ve bu işlem uzun sürdüğü için kopyası alındı.
df_ = pd.read_excel("/Users/aslihankalyonkat/Desktop/DSMLBC/datasets/online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom


In [4]:
#Veri setindeki eksik gözlemler
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [5]:
#Eksik gözlemler kaldırılıyor
df.dropna(inplace=True)

In [6]:
#Hangi üründen kaçar tane kayıt var
df["Description"].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER     2070
REGENCY CAKESTAND 3 TIER               1905
JUMBO BAG RED RETROSPOT                1662
ASSORTED COLOUR BIRD ORNAMENT          1418
PARTY BUNTING                          1416
                                       ... 
NEW BAROQUE LARGE NECKLACE BLK/WHIT       1
POTTING SHED SOW 'N' GROW SET             1
FOOD COVER WITH BEADS , SET 2 SIZES       1
GLASS BEAD HOOP EARRINGS BLACK            1
EASTER CRAFT IVY WREATH WITH CHICK        1
Name: Description, Length: 3896, dtype: int64

In [7]:
#En çok sipariş edilen 5 ürün
df.groupby('Quantity').sum().sort_values(by='Quantity', ascending=False).head()

,Price,Customer ID
Quantity,,
80995,2.08,16446.00
74215,1.04,12346.00
12540,0.00,13256.00
4800,0.21,12901.00
4300,0.72,13135.00


In [8]:
#Toplam kaç adet fatura kesildiği bilgisi
df["Invoice"].nunique()

22190

In [9]:
#Faturalardaki ‘C’ iptal edilen işlemleri göstermektedir. İptal edilen işlemlerin veri setinden çıkartılması
df = df[~df['Invoice'].str.contains('C', na=False)]
df["Invoice"].nunique()

18536

In [10]:
# Fatura başına elde edilen toplam kazancı ifade eden ‘TotalPrice’ adında bir değişken oluşturulması
df['TotalPrice'] = df['Quantity'] * df['Price']
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34


In [11]:
# Fatura numaralarına göre gruplayarak, faturalardan elde edilen kazanç görülebilir
df.groupby("Invoice").agg({"TotalPrice":"sum"}).head()

,TotalPrice
Invoice,
536365,139.12
536366,22.20
536367,278.73
536368,70.05
536369,17.85


# RFM Metriklerinin Hesaplanması

In [12]:
# Bu veri seti için analizin yapıldığı tarih 11.12.2011 seçilmiştir.
today_date = dt.datetime(2011, 12, 11)

# Recency, Frequency ve Monetary metriklerinin hesaplanması
# Recency: Müşterinin son alışveriş yaptığı tarihin üzerinden geçen gün
# Frequency: Müşterinin alışveriş sayısı
# Monetary: Müşterinin toplam kazandırdığı miktar

rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [13]:
#Oluşturulan RFM metrikleri için sütun isimlerinin düzenlenmesi
rfm.columns = ['recency', 'frequency', 'monetary']
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12346.00,326,1,77183.60
12347.00,3,7,4310.00
12348.00,76,4,1797.24
12349.00,19,1,1757.55
12350.00,311,1,334.40


In [14]:
# RFM Metriklerinin anlamlı yorumlanabilmesi için 1-5 aralığında skorlara çevirmeliyiz
rfm['receny_score'] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['frequency_score'] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm['monetary_score'] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

rfm.head()

,recency,frequency,monetary,receny_score,frequency_score,monetary_score
Customer ID,,,,,,
12346.00,326,1,77183.60,1,1,5
12347.00,3,7,4310.00,5,5,5
12348.00,76,4,1797.24,2,4,4
12349.00,19,1,1757.55,4,1,4
12350.00,311,1,334.40,1,1,2


In [15]:
# RFM skorlarından faydalanarak segmentlere ayırmak için RFM_SCORE sütunu oluşturuluyor.
# Bu analizde recency ve frequency skorlarından faydalanılacak.

rfm['RFM_SCORE'] = (rfm['receny_score'].astype(str) +
                    rfm['frequency_score'].astype(str))

rfm.head()

,recency,frequency,monetary,receny_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.00,326,1,77183.60,1,1,5,11
12347.00,3,7,4310.00,5,5,5,55
12348.00,76,4,1797.24,2,4,4,24
12349.00,19,1,1757.55,4,1,4,41
12350.00,311,1,334.40,1,1,2,11


# RFM Segmentasyon Tablosu
<br>
Bu tablodaki değerlerden faydalanarak RFM_SCORE değerlerine göre segmentlere ayıracağız.
<br>

![title](https://guillaume-martin.github.io/images/rfm-segments.png)

In [16]:
# Tablodaki segmentler için map tanımlandı
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [17]:
# RFM_SCORE değerleri ve seg_map'den faydalanılarak segment sütunu oluşturulur.
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm.head()

,recency,frequency,monetary,receny_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.00,326,1,77183.60,1,1,5,11,hibernating
12347.00,3,7,4310.00,5,5,5,55,champions
12348.00,76,4,1797.24,2,4,4,24,at_risk
12349.00,19,1,1757.55,4,1,4,41,promising
12350.00,311,1,334.40,1,1,2,11,hibernating


In [18]:
# Segmentler hakkında genel bilgiler
rfm[['segment', 'recency', 'frequency', 'monetary']].groupby('segment').agg(['mean', 'count', 'max', 'sum']).T

segment          about_to_sleep   at_risk  cant_loose  champions  hibernating  \
recency   mean            53.31    153.79      132.97       6.36       217.61   
          count          352.00    593.00       63.00     633.00      1071.00   
          max             72.00    374.00      373.00      13.00       374.00   
          sum          18766.00  91195.00     8377.00    4027.00    233055.00   
frequency mean             1.16      2.88        8.38      12.42         1.10   
          count          352.00    593.00       63.00     633.00      1071.00   
          max              2.00      6.00       34.00     210.00         2.00   
          sum            409.00   1707.00      528.00    7860.00      1180.00   
monetary  mean           471.99   1084.54     2796.16    6857.96       488.64   
          count          352.00    593.00       63.00     633.00      1071.00   
          max           6207.67  44534.30    10254.18  280206.02     77183.60   
          sum         166142.02 643129.43   176157.82 4341091.16    523336.98   

segment          loyal_customers  need_attention  new_customers  \
recency   mean             33.61           52.43           7.43   
          count           819.00          187.00          42.00   
          max              72.00           72.00          13.00   
          sum           27525.00         9804.00         312.00   
frequency mean              6.48            2.33           1.00   
          count           819.00          187.00          42.00   
          max              63.00            3.00           1.00   
          sum            5307.00          435.00          42.00   
monetary  mean           2864.25          897.63         388.21   
          count           819.00          187.00          42.00   
          max          124914.53        12601.83        3861.00   
          sum         2345818.94       167856.41       16304.94   

segment          potential_loyalists  promising  
recency   mean                 17.40      23.42  
          count               484.00      95.00  
          max                  33.00      33.00  
          sum                8421.00    2225.00  
frequency mean                  2.01       1.00  
          count               484.00      95.00  
          max                   3.00       1.00  
          sum                 973.00      95.00  
monetary  mean               1041.22     290.91  
          count               484.00      95.00  
          max              168472.50    1757.55  
          sum              503951.45   27636.75

# Aksiyon Planı ve Yorumlar

**Champions** <br>
- Bu segmentte 633 kişi bulunmakta, <br>
- Ortalama olarak en son alışverişleri 6 gün önce gerçekleşmiş, <br>
- Alışveriş sıklıkları 12, toplamda 7860 alışverişleri var, <br>
- Bu segmente en fazla harcama yapan müşteri 280206.02 sterlin harcama yapmış ve segmentin toplam 4341091.16 sterlin harcama gerçekleştirmiştir.
<br><br>
-> Bu segmentdeki müşterilerimiz en önemli müşterilerdir. Bundan dolayı onlara kendilerini özel hissetirecek hareketler yapmalıyız. Yeni çıkacak bir ürünü ilk onların beğenisine sunabiliriz. Doğum günü gibi özel günlerinde indirim kuponu yollayabiliriz.
<br><br>

**At_Risk** <br> 
- Bu segmentte 593 kişi bulunmakta, <br>
- Ortalama olarak en son alışverişleri 154 gün önce gerçekleşmiş, <br>
- Alışveriş sıklıkları 3, toplamda 1707 alışverişleri var, <br>
- Bu segmente en fazla harcama yapan müşteri 44534.30 sterlin harcama yapmış ve segmentin toplam 643129.43 sterlin harcama gerçekleştirmiştir.
<br><br>
-> Önem verilmesi gereken segmentlerden birisidir. Çünkü bu segmentte kaybetme riskiyle karşı karşıya olduğumuz 593 kişi bulunmaktadır. Mail veya sms gibi iletişim araçlarını kullanarak onları hareketlendirebiliriz. Örneğin; bu haftaya özel yapacağınız harcamalara şu kadar puan kazanacaksınız.
<br><br>

**Hibernating**<br>
- Bu segmentte 1071 kişi bulunmakta,<br>
- Ortalama olarak en son alışverişleri 218 gün önce gerçekleşmiş,<br>
- Alışveriş sıklıkları 1, toplamda 1180 alışverişleri var,<br>
- Bu segmente en fazla harcama yapan müşteri 77183.60 sterlin harcama yapmış ve segmentin toplam 523336.98 sterlin harcama gerçekleştirmiştir.
<br><br>
-> Bu segmentteki müşterilerimize kampanya mailleri yollayarak veya yeni çıkan ürünlerin haberini vererek dikkatlerini çekip geri kazanmayı amaçlamalıyız.
<br><br>

In [19]:
#İhtiyaç durumunda herhangi bir segmentin excel olarak çıktısının alınması.
#Loyal_customers segmentinin çıktısını alarak sadık müşterilerimizin kim olduğunu görebiliriz.

rfm.reset_index(inplace=True)
rfm[rfm['segment'] == 'loyal_customers']['Customer ID'].to_excel("loyal_customers.xlsx")